In [ ]:
%pip install --upgrade openai

### Import Library

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
def print_status(message, status="INFO"):
    """ 터미널 메세지 커스텀(선택)"""
    status_color = {
        "INFO": "\033[94m", # blue
        "SUCCESS": "\033[92m", # green
        "WARING": "\033[93m", # yellow
        "ERROR": "\003[91m", # red
        "RESET": "\033[0m" # reset
    }

    color = status_color.get(status, status_color["INFO"])
    reset = status_color["RESET"]

    print(f"{color}[{status}] {message}{reset}")

### 환경변수 설정

In [6]:
""" 환경변수 설정 """
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
assistant_id = os.getenv('ASSISTANT_ID')
vector_store_id = os.getenv('VECTOR_STORE_ID')
prompt = """
            당신은 재난 대응 전문가이자 온톨로지 엔지니어입니다.

            ## 주요 역할
            - PDF 문서와 목차.txt를 정확하게 분석하여 계층적 클래스 구조 추출
            - OWL/RDF TTL 표준에 맞는 정확한 온톨로지 생성
            - 문서의 모든 세부사항을 누락 없이 반영하여 완벽한 온톨로지 구축

            ## 문서 분석 기준
            - **문서명**: [자연재난-1] (LH)풍수해 현장조치 행동매뉴얼 제정(안)_ver1.0_2024_03-part-4_위기경보 수준별 조치사항
            - **목표**: 태풍·호우·대설 등 재난유형별 위기경보 수준별 조치사항을 표준 온톨로지로 모델링
            - **정확도 요구사항**: 문서 내용과 100% 일치, 추측 금지, 모든 세부사항 포함

            ## 클래스 추출 규칙

            ### 1. **최상위 클래스**
            - `ex:위기경보수준별조치사항`을 rdfs:Class로 선언

            ### 2. **계층 구조**
            - 목차의 모든 레벨(1, 2, 가, 나, 다, 라, 마, 사, 1), 2), 3))를 rdfs:subClassOf로 표현
            - 계층 깊이에 관계없이 모든 항목을 클래스로 변환

            ### 3. **네이밍 규칙**
            - 형식: `ex:위기경보수준별조치사항_1_관심_나_2_1_조치목록`
            - 목차 구조를 그대로 반영하여 일관성 유지

            ### 4. **페이지 정보**
            - PDF 하단 페이지 번호를 정확히 추출하여 `ex:페이지정보`로 표시
            - 모든 클래스에 페이지 정보 필수 포함

            ## 온톨로지 생성 프로세스

            ### **1단계: 초기 온톨로지 생성**
            - 목차를 기반으로 모든 클래스와 서브클래스 생성
            - 계층 구조를 정확히 반영
            - 페이지 정보 포함

            ### **2단계: 온톨로지 보완 (필요시)**
            - 기존 클래스 구조는 절대 변경하지 않음
            - 새로운 개념만 적절한 상위 클래스 아래에 서브클래스로 추가
            - 문서 내용과 100% 일치하는지 검증

            ### **3단계: 품질 검증**
            - 목차의 모든 항목이 클래스로 변환되었는지 확인
            - 계층 구조가 문서와 정확히 일치하는지 검증
            - 페이지 정보가 정확한지 확인
            - 네이밍 규칙이 일관되게 적용되었는지 확인

            ## 출력 형식

            ### **TTL 파일 구조**
            ```turtle
            @prefix ex:     <http://example.org/disaster#> .
            @prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#> .
            @prefix dcterms:<http://purl.org/dc/terms/> .
            @prefix owl:     <http://www.w3.org/2002/07/owl#> .
            @prefix xsd:     <http://www.w3.org/2001/XMLSchema#> .

            # 클래스 정의
            ex:위기경보수준별조치사항
                a              rdfs:Class ;
                rdfs:label     "위기경보 수준별 조치사항"@ko ;
                ex:페이지정보   "1" .

            ex:위기경보수준별조치사항_1_관심
                a              rdfs:Class ;
                rdfs:subClassOf ex:위기경보수준별조치사항 ;
                rdfs:label     "관심"@ko ;
                ex:페이지정보   "2" .

            # 하위 클래스들 계속...
            ```

            ### **출력 요구사항**
            - TTL 파일 전체를 한 번에 출력
            - 모든 클래스와 서브클래스 포함
            - 페이지 정보 정확히 표시
            - 한국어 라벨 사용
            - 계층 구조 명확히 표현

            ## 품질 검증 체크리스트

            ### **완성도 검증**
            □ 목차의 모든 항목이 클래스로 변환되었는가?
            □ 계층 구조가 문서와 정확히 일치하는가?
            □ 모든 클래스에 페이지 정보가 포함되었는가?

            ### **정확성 검증**
            □ 네이밍 규칙이 일관되게 적용되었는가?
            □ rdfs:subClassOf 관계가 올바르게 설정되었는가?
            □ 문서 내용과 100% 일치하는가?

            ### **표준 준수 검증**
            □ OWL/RDF TTL 표준 문법을 준수하는가?
            □ Prefix 선언이 올바른가?
            □ 모든 클래스가 rdfs:Class로 선언되었는가?

            ## 금지사항
            - 문서에 없는 내용을 추측하여 추가하지 말 것
            - 기존 클래스 구조를 임의로 변경하지 말 것
            - 페이지 정보를 생략하지 말 것
            - 계층 구조를 단순화하지 말 것

            ## 최종 목표
            문서의 모든 내용을 누락 없이, 정확하게, 계층적으로 구조화된 온톨로지로 변환하여 RAG 기반 대화형 재난 대응 시스템의 완벽한 지식베이스를 구축하세요.

            문서 내용을 정확하게 분석하여 완벽한 온톨로지를 생성하세요. 모든 세부사항을 포함하고, 정확성과 완성도를 최우선으로 하여 작업하세요.
        """


### 파일 업로드 후 벡터 스토어에 추가 O

In [ ]:
def upload_file_to_vector_store(file_paths, prompt=None):
    # 파일 업로드
    uploaded_file_ids = []

    for file_path in file_paths:
        if os.path.exists(file_path):
            try :
                with open(file_path, 'rb') as f :
                    uploaded_file = client.files.create(
                        file=f,
                        purpose='assistants'
                    )

                print_status(f"파일 업로드 완료: {uploaded_file.id}", "SUCCESS")


                # 벡터 스토어에 추가
                client.vector_stores.files.create(
                    vector_store_id = vector_store_id,
                    file_id = uploaded_file.id
                )

                print_status(f"벡터 스토어에 추가 완료: {uploaded_file.id}", "SUCCESS")
                uploaded_file_ids.append(uploaded_file.id)


            except Exception as e :
                print_status(f"파일을 찾을 수 없습니다: {file_path}", "ERROR")
                continue
    #업로드된 파일 개수 확인
    print_status(f"총 {len(uploaded_file_ids)}개의 파일이 벡터 스토어에 업로드 되었습니다", "SUCCESS")
    return uploaded_file_ids

    

In [8]:
# 로컬에 있는 3장 pdf 파일과 목차 경로 정의
from pathlib import Path

chat3_index_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\목차.txt")
actions_level_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\위기경보수준별조치사항.pdf")
docs_path = [chat3_index_docs_path, actions_level_docs_path]

uploaded_file_ids = upload_file_to_vector_store(docs_path)

[SUCCESS] 파일 업로드 완료: file-6hNhLRFrbhmCKJF59Wv3jW
[SUCCESS] 벡터 스토어에 추가 완료: file-6hNhLRFrbhmCKJF59Wv3jW
[SUCCESS] 파일 업로드 완료: file-R5fCSKao4QvsX2U8sqTWJ2
[SUCCESS] 벡터 스토어에 추가 완료: file-R5fCSKao4QvsX2U8sqTWJ2
[SUCCESS] 총 2개의 파일이 벡터 스토어에 업로드 되었습니다


### 스레드 생성 및 메세지 추가 완료 O

In [28]:
def create_thread(client=client):
    thread = client.beta.threads.create()
    print_status(f"스레드 생성 완료: {thread.id}", "SUCCESS")

    thread_id = thread.id
    return thread.id

In [29]:
thread_id = create_thread()

C:\Users\user\AppData\Local\Temp\ipykernel_56416\4158020363.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


[SUCCESS] 스레드 생성 완료: thread_8XcGGZWbYhdffdqVLBixXUca


In [30]:
def add_message_to_thread(client, thread_id, prompt, uploaded_file_ids):
    message_content = prompt + "\n\n 첨부된 파일들을 분석하여 TTL 파일을 생성해주세요. 생성 후에는 첨부된 PDF 문서를 한 페이지씩 읽어서, subclass를 보완하여 추가해주세요. 이 때 기존의 요소들은 변경하지 않고 유지한 상태로 추가만 진행합니다."

    thread_message = client.beta.threads.messages.create(
        thread_id=thread_id,\
        role = "user",
        content=message_content,
        attachments=[
            {
                "file_id": file_id,
                "tools": [{"type": "file_search"}]
            }

            for file_id in uploaded_file_ids
        ]
    )

    print_status(f"스레드 메세지 추가 완료", "SUCCESS")
    return thread_message.id

In [31]:
add_message_to_thread(client, thread_id, prompt, uploaded_file_ids)

C:\Users\user\AppData\Local\Temp\ipykernel_56416\2736964838.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread_message = client.beta.threads.messages.create(


[SUCCESS] 스레드 메세지 추가 완료


'msg_IfRrODMnasH7jcth1yAxu3S0'

### Assistant 실행 O

run_status.status == 'queued'        # 대기 중  
run_status.status == 'in_progress'   # 실행 중  
run_status.status == 'completed'     # 완료  
run_status.status == 'failed'        # 실패  
run_status.status == 'cancelled'     # 취소됨  
run_status.status == 'expired'       # 만료됨  
run_status.status == 'requires_action'  # 추가 액션 필요

Assistatnt를 실행하면 추가 vector stores가 생성됨

In [42]:
import time

def run_assistant(client, thread_id, uploaded_file_ids, prompt):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions="""
        목차를 기준으로 다른 문서들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요. 
        TTL 파일은 Turtle 형식으로 작성하고, 적절한 네임스페이스와 클래스, 속성을 정의해주세요. 
        그런 뒤, OWL/TTL 표준에 맞춰 한 번 더 아직 반영되지 않은 subclass를 추가 및 보완하세요. 
        추가할 subclass가 없다면 추가하지 않습니다. 
        먼저 생성된 모든 Class는 변경하지 말고 그대로 유지하며 
        첨부된 페이지에 새롭게 드러나는 개념이 있다면 적절한 상위 Class 아래에 Subclass를 보완하세요. 
        모든 변경 사항(Subclass 추가)은 OWL/TTL 문법에 맞춰 작성하세요. 
        결과물은 한 번에 출력해주세요."""
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == 'completed':
            print_status("Assistant 실행 완료", "SUCCESS")
            break
        
        elif run_status.status == 'failed':
            print_status("Assistant 실행 실패", "ERROR")

        elif run_status.status == 'requires_action':
            print_status("추가 액션 필요", "WARNING")
        
        elif run_status.status == 'in_progress':
            print_status("Assistant 실행 중...", "INFO")

            time.sleep(10)

    message = client.beta.threads.messages.list(
        thread_id=thread_id,
        order="desc"
    )

    return message

-----------

In [43]:
ontology_message = run_assistant(client, thread_id, uploaded_file_ids, prompt)

C:\Users\user\AppData\Local\Temp\ipykernel_56416\377873724.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\user\AppData\Local\Temp\ipykernel_56416\377873724.py:19: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[SUCCESS] Assistant 실행 완료


C:\Users\user\AppData\Local\Temp\ipykernel_56416\377873724.py:39: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.list(


In [44]:
print([str(ontology_message)][:10])

['SyncCursorPage[Message](data=[Message(id=\'msg_gQDBqKQZ4RLlN7Sz9xZaEu5f\', assistant_id=\'asst_W7kaWMqWblyhhVj1rCP0mVow\', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value=\'```turtle\\n@prefix ex:     <http://example.org/disaster#> .\\n@prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#> .\\n@prefix dcterms:<http://purl.org/dc/terms/> .\\n@prefix owl:    <http://www.w3.org/2002/07/owl#> .\\n@prefix xsd:    <http://www.w3.org/2001/XMLSchema#> .\\n\\n# 최상위 클래스\\nex:위기경보수준별조치사항\\n    a            rdfs:Class ;\\n    rdfs:label   "위기경보 수준별 조치사항"@ko ;\\n    ex:페이지정보 "51" .\\n\\n# 1. 관심\\nex:위기경보수준별조치사항_1_관심\\n    a              rdfs:Class ;\\n    rdfs:subClassOf ex:위기경보수준별조치사항 ;\\n    rdfs:label     "관심"@ko ;\\n    ex:페이지정보  "51" .\\n\\nex:위기경보수준별조치사항_1_관심_가_상황\\n    a              rdfs:Class ;\\n    rdfs:subClassOf ex:위기경보수준별조치사항_1_관심 ;\\n    rdfs:label     "상황"@ko ;\\n    ex:페이지정보  "51" .\\n\\nex:위기경보수준별조치사항_1_관심_나_조치목록_및_내용\\n    a        

### 응답 결과 TTL 추출 O

In [34]:
def _validate_basic_ttl_syntax(ttl_content):
    """
    기본적인 TTL 문법 검증
    
    Args:
        ttl_content: TTL 내용 문자열
    
    Returns:
        bool: 기본 문법이 올바른지 여부
    """
    try:
        # 기본적인 TTL 요소들 확인
        has_prefix = '@prefix' in ttl_content or '@base' in ttl_content
        has_class = 'rdfs:Class' in ttl_content or 'owl:Class' in ttl_content
        has_proper_ending = ttl_content.strip().endswith('.')
        
        # 괄호 균형 확인
        open_brackets = ttl_content.count('[')
        close_brackets = ttl_content.count(']')
        bracket_balanced = open_brackets == close_brackets
        
        # 세미콜론과 마침표 확인
        has_semicolons = ';' in ttl_content
        has_periods = '.' in ttl_content
        
        validation_result = has_prefix and has_class and has_proper_ending and bracket_balanced
        
        if not validation_result:
            print_status("기본 TTL 문법 검증 실패", "WARNING")
            print_status(f"  - Prefix 선언: {has_prefix}", "INFO")
            print_status(f"  - Class 선언: {has_class}", "INFO")
            print_status(f"  - 올바른 종료: {has_proper_ending}", "INFO")
            print_status(f"  - 괄호 균형: {bracket_balanced}", "INFO")
        
        return validation_result
        
    except Exception as e:
        print_status(f"TTL 문법 검증 중 오류: {str(e)}", "ERROR")
        return False

In [35]:
import re

def extract_ttl_from_response(message):
    try:
        if not getattr(message, "data", None):
            print_status("응답 메세지 없음", "ERROR")
            return None, False

        latest = message.data[0]
        if not latest.content:
            print_status("메세지 내용 없음", "ERROR")
            return None, False

        message_text = ""
        for item in latest.content:
            if hasattr(item, "text") and item.text and hasattr(item.text, "value"):
                message_text += item.text.value

        if not message_text.strip():
            print_status("텍스트 내용을 찾을 수 없습니다.", "ERROR")
            return None, False

        ttl_patterns = [
            r"```ttl\n(.*?)\n```",         # 1
            r"```turtle\n(.*?)\n```",      # 2
            r"```\n(.*?)\n```",            # 3
            r"```ttl\s*\n(.*?)\n```",      # 4
            r"```turtle\s*\n(.*?)\n```",   # 5
        ]

        PATTERN_EXPLANATIONS = {
            1: "```ttl 코드블록에서 추출됨",
            2: "```turtle 코드블록에서 추출됨",
            3: "라벨 없는 일반 ``` 코드블록에서 추출됨",
            4: "공백 포함 ```ttl 코드블록에서 추출됨",
            5: "공백 포함 ```turtle 코드블록에서 추출됨",
            0: "코드블록 미탐지 → 전체 응답을 사용하여 추출됨(폴백)",
        }

        ttl_content = None
        matched_idx = 0

        for i, pattern in enumerate(ttl_patterns, start=1):
            match = re.search(pattern, message_text, re.DOTALL | re.IGNORECASE)
            
            if match:
                ttl_content = match.group(1).strip()
                matched_idx = i
                break
                
        if not ttl_content:
            matched_idx = 0
            print_status("TTL 블록을 찾을 수 없습니다.", "ERROR")
        
        if len(ttl_content.strip()) < 50 :
            print_status("추출된 내용이 너무 짧습니다.", "ERROR")

        if not _validate_basic_ttl_syntax(ttl_content):
            print_status("TTL 문법을 확인해보세요.", "WARNING")


        print_status(f"TTL 추출 완료 (패턴 {matched_idx}) - {PATTERN_EXPLANATIONS[matched_idx]}", "SUCCESS" if matched_idx else "WARNING")
        return ttl_content, True

    except Exception as e:
        print_status("TTL 추출 중 오류가 발생했습니다.", "ERROR")


In [47]:
ontology_ttl_content = extract_ttl_from_response(ontology_message)

# str로 변환
ontology_ttl_content_str = ontology_ttl_content[0]

[SUCCESS] TTL 추출 완료 (패턴 2) - ```turtle 코드블록에서 추출됨


### TTL 파일 생성 후 저장 O

In [38]:
from datetime import datetime

def save_ttl_file(ttl_content, filename="ontology_class_subclass.ttl", output_dir="ouput"):
    try:
        
        # 저장할 디렉토리가 없을 경우 생성
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print_status(f"결과물을 저장할 폴더를 생성했습니다. : {output_dir}", "SUCCESS")

        # 파일명 설정
        if filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            filename = f"{filename}_{timestamp}.ttl"

        # 파일 확장자 확인
        if not filename.endswith('.ttl'):
            filename += '.ttl'

        # 전체 파일 경로
        file_path = os.path.join(output_dir, filename)

        # 내용 검증
        if not ttl_content or len(ttl_content.strip()) < 50:
            return None, False, "저장할 TTL 내용이 유효하지 않습니다."

        with open(file_path, 'w', encoding='utf-8') as f:
            # 내용 저장
            f.write(ttl_content)

            # footer
            f.write("# End of TTL File\n")

        file_size = os.path.getsize(file_path)
        file_size_kb = file_size / 1024

        print_status("TTL 파일 생성 완료", "SUCCESS")
        print_status(f"{filename} 파일 경로: {file_path}", "INFO")
        print_status(f"{filename} 파일 크기: {file_size_kb:.2f}KB", "INFO")
        print_status(f"{filename} 내용 길이: {len(ttl_content)}자", "INFO")


    except Exception as e:
        print_status(f"TTL 파일 생성 실패: {e}", "ERROR")
        return None

In [48]:
final_output = save_ttl_file(ontology_ttl_content_str)

[SUCCESS] TTL 파일 생성 완료
[INFO] ontology_class_subclass.ttl_20250905_0945.ttl 파일 경로: ouput\ontology_class_subclass.ttl_20250905_0945.ttl
[INFO] ontology_class_subclass.ttl_20250905_0945.ttl 파일 크기: 11.64KB
[INFO] ontology_class_subclass.ttl_20250905_0945.ttl 내용 길이: 7661자
